In [23]:
# penguin_nn_logistic_equivalent.py
import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [24]:
# Load the penguins dataset
penguins = sns.load_dataset('penguins')
# Drop rows with missing values
penguins.dropna(inplace=True)
# Features
X = penguins[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']]
# Target: 1 if Adelie, 0 otherwise
y = (penguins['species'] == 'Adelie').astype(int)

In [25]:
zero_init = tf.keras.initializers.Zeros()

tf_model = tf.keras.Sequential([
    tf.keras.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(
        1,
        activation='sigmoid',
        kernel_initializer=zero_init,
        bias_initializer=zero_init
    )
])

# SGD optimizer (small learning rate, no momentum)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

In [26]:
# Compile model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [29]:
# Train model
model.fit(X_train, y_train, epochs=100, verbose=0)

In [30]:
# Extract weights and bias
weights, bias = model.layers[0].get_weights()
print("\nNeural Network Weights (equivalent to logistic regression coefficients):")
for feature, w in zip(['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'], weights.flatten()):
    print(f"{feature}: {w:.4f}")
print(f"Bias (intercept): {bias[0]:.4f}")


Neural Network Weights (equivalent to logistic regression coefficients):
bill_length_mm: -0.9188
bill_depth_mm: 2.7881
flipper_length_mm: 5.2734
body_mass_g: -2.6423
Bias (intercept): 1.4476
